# Problem set 3: Text analysis of DOJ press releases

**Total points (without extra credit)**: 52 

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases

## 0.0 Import packages

In [1]:
## helpful packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
### nltk.download('averaged_perceptron_tagger')
### nltk.download('stopwords')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
### ! python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

## 0.1 Load and clean text data

In [2]:
## first, unzip the file pset3_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values
doj = pd.read_json("combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()

id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (17 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [3]:
## your code to subset to one press release and take the string

pharma = doj.loc[doj['id'] == '17-1204', 'contents'].to_string(index=False)
pharma

'The founder and majority owner of Insys Therapeutics Inc., was arrested today and charged with leading a nationwide conspiracy to profit by using bribes and fraud to cause the illegal distribution of a Fentanyl spray intended for cancer patients experiencing breakthrough pain.\xa0"More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids. And yet some medical professionals would rather take advantage of the addicts than try to help them," said Attorney General Jeff Sessions. "This Justice Department will not tolerate this.\xa0 We will hold accountable anyone – from street dealers to corporate executives -- who illegally contributes to this nationwide epidemic.\xa0 And under the leadership of President Trump, we are fully committed to defeating this threat to the American people.”John N. Kapoor, 74, of Phoenix, Ariz., a current member of the Board of Directors of Insys, was arrested this morning in Arizona and charged with RICO conspi

### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (you can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for `.isalpha()`: https://www.w3schools.com/python/ref_string_isalpha.asp

In [4]:
#tokenize the text
tokens = word_tokenize(pharma)

#A
#remove all punc/digits
alpha_tokens = [token for token in tokens if token.isalpha()]


In [5]:
#B
#uses speech tagger 
pos_tags = pos_tag(alpha_tokens)


#use code from class to extract adj
all_adj_noun = [one_tok[0] for one_tok in pos_tags
                if one_tok[1] == "JJ" or
                one_tok[1] == "JJR" or one_tok[1] == "JJS"]

#C
#create a pandas Series from the list of adjectives
adjective_series = pd.Series(all_adj_noun)

#count occurrences of each adjective
adjective_counts = adjective_series.value_counts()

#convert the Series into a df and rename the index and column
df_adjectives = adjective_counts.reset_index()
df_adjectives.columns = ['Adjective', 'Count']

#get the top 5 most frequent adjectives
top_adjectives = df_adjectives.head(5)

top_adjectives

,Adjective,Count
0,former,8
1,opioid,5
2,nationwide,4
3,addictive,3
4,other,3


## 1.2 named entity recognition (4 points)

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`

In [6]:
# A
#extract all named entities
spacy_pharma = nlp(pharma)

for one_tok in spacy_pharma.ents:
    print("Entity: " + one_tok.text + "; NER tag: " + one_tok.label_)

Entity: Insys Therapeutics Inc.; NER tag: ORG
Entity: today; NER tag: DATE
Entity: Fentanyl; NER tag: PERSON
Entity: More than 20,000; NER tag: CARDINAL
Entity: Americans; NER tag: NORP
Entity: last year; NER tag: DATE
Entity: millions; NER tag: CARDINAL
Entity: Jeff Sessions; NER tag: PERSON
Entity: This Justice Department; NER tag: ORG
Entity: Trump; NER tag: PERSON
Entity: American; NER tag: NORP
Entity: ”John N. Kapoor; NER tag: PERSON
Entity: 74; NER tag: DATE
Entity: Phoenix; NER tag: GPE
Entity: Ariz.; NER tag: GPE
Entity: the Board of Directors; NER tag: ORG
Entity: Insys; NER tag: ORG
Entity: this morning; NER tag: TIME
Entity: Arizona; NER tag: GPE
Entity: RICO; NER tag: LAW
Entity: Kapoor; NER tag: PERSON
Entity: Executive; NER tag: ORG
Entity: Board; NER tag: ORG
Entity: Insys; NER tag: ORG
Entity: Phoenix; NER tag: GPE
Entity: today; NER tag: DATE
Entity: U.S.; NER tag: GPE
Entity: District Court; NER tag: ORG
Entity: Boston; NER tag: GPE
Entity: a later date; NER tag: DAT

In [7]:
# B
#create an empty set
law_entities = set()

#check if each entity is law
for entity in spacy_pharma.ents:
    if entity.label_ == 'LAW':
        law_entities.add(entity.text + ": " + entity.label_)

# print entities with LAW tag
for item in law_entities:
    print(item)

RICO: LAW
the Controlled Substances Act: LAW


C. Use Google to summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case (and not just a mafia case...) 

**Summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case**


RICO stands for "Racketeer Influenced and Corrupt Organizations Act," and is a federal law which sets guidelines for who can be charged with racketeering based on their past activity. RICO could apply to a pharmaceutical kickback case because there could be a coordinated effort by pharma people to defraud consumers, the SEC, etc. with the profits they are making or the drugs they have produced (i.e. they are engaging in unlawful behavior with racketeering or fraud).

D. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these named entities.

In [8]:
# Regex to find 'year' or 'years' in the entity text
year_regex = re.compile(r'\b(year|years)\b', re.IGNORECASE)

# List to hold relevant DATE entities
relevant_dates = []

# Filtering entities
for ent in spacy_pharma.ents:
    if ent.label_ == "DATE" and year_regex.search(ent.text):
        relevant_dates.append(ent.text + ": " + ent.label_)

# Printing the filtered entities
for date in relevant_dates:
    print(date)

last year: DATE
20 years: DATE
three years: DATE
five years: DATE
three years: DATE


E. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

**Hint**: you may want to use re.search or re.findall 

- For part E, you can use `re.search` and `re.findall`, or anything that works 😳.

In [9]:
## your code here

sentences_with_years = []

for ent in spacy_pharma.ents:
    if ent.label_ == "DATE" and year_regex.search(ent.text):
        # Add the whole sentence containing the entity to the list
        sentences_with_years.append(ent.sent.text.strip())

# Print the relevant sentences
for sentence in sentences_with_years:
    print("Relevant Sentence:", sentence)

Relevant Sentence: "More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids.
Relevant Sentence: The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss.
Relevant Sentence: The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss.
Relevant Sentence: The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised release and a $25,000 fine.
Relevant Sentence: The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater than

In [ ]:
# hmm first sentence doesn't seem relevant? 

# CHECK! above

**Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum).**


The CEO is looking at a maximum of 20 years in prison and 3 years of supervised released. There are also associated fines with the charge,

## 1.3 sentiment analysis  (10 points)

A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [10]:
##creating the subset
topics_of_interest = ["Civil Rights", "Hate Crimes", "Project Safe Childhood"]
doj_subset = doj[doj['topics_clean'].isin(topics_of_interest)]

#test the number of rows
print("Number of rows in the subset:", len(doj_subset))

Number of rows in the subset: 717


B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint**: you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- A function + list comprehension to execute will takes about 30 seconds on a respectable local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [12]:
# Initialize scorer
sent_obj = SentimentIntensityAnalyzer()

def process_press(text):
    # Generate named entities
    spacy_text = nlp(text)

    # Remove named entities
    for entity in spacy_text.ents:
        text = text.replace(entity.text, "")

    # Conduct sentiment analysis
    sentiment_ex = sent_obj.polarity_scores(text)

    return sentiment_ex

In [14]:
sentiment_scores = [process_press(str) for str in doj_subset['contents']]
sentiment_scores

[{'neg': 0.2, 'neu': 0.751, 'pos': 0.049, 'compound': -0.9931},
 {'neg': 0.129, 'neu': 0.804, 'pos': 0.066, 'compound': -0.9325},
 {'neg': 0.09, 'neu': 0.835, 'pos': 0.075, 'compound': -0.7579},
 {'neg': 0.121, 'neu': 0.798, 'pos': 0.081, 'compound': -0.9037},
 {'neg': 0.175, 'neu': 0.782, 'pos': 0.043, 'compound': -0.9864},
 {'neg': 0.148, 'neu': 0.8, 'pos': 0.053, 'compound': -0.987},
 {'neg': 0.146, 'neu': 0.78, 'pos': 0.074, 'compound': -0.9559},
 {'neg': 0.09, 'neu': 0.847, 'pos': 0.063, 'compound': -0.7783},
 {'neg': 0.103, 'neu': 0.838, 'pos': 0.059, 'compound': -0.9136},
 {'neg': 0.162, 'neu': 0.784, 'pos': 0.055, 'compound': -0.9801},
 {'neg': 0.216, 'neu': 0.748, 'pos': 0.036, 'compound': -0.9973},
 {'neg': 0.091, 'neu': 0.847, 'pos': 0.061, 'compound': -0.8519},
 {'neg': 0.08, 'neu': 0.857, 'pos': 0.063, 'compound': -0.6486},
 {'neg': 0.319, 'neu': 0.643, 'pos': 0.038, 'compound': -0.995},
 {'neg': 0.175, 'neu': 0.757, 'pos': 0.067, 'compound': -0.9889},
 {'neg': 0.121, 'neu

C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score and print the top `id`, `contents`, and `neg` columns of the two most neg press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [15]:
# Create new data frame of sentiment scores
sentiment = pd.DataFrame(sentiment_scores, columns = ['neg', 'neu', 'pos', 'compound'])

# Reset indices of sentiment and doj_subset
doj_subset = doj_subset.reset_index()
sentiment = sentiment.reset_index()

# Concatenate sentiment and doj_subset data frames
doj_subset_wscores = pd.concat([doj_subset, sentiment], axis = 1)

# Sort values from highest to lowest neg score
doj_subset_wscores = doj_subset_wscores.sort_values(by = 'neg', ascending = False)

# Print top two most negative press releases
top_neg = doj_subset_wscores.head(2)
top_neg[['id', 'contents', 'neg']]

,id,contents,neg
13,14-248,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",0.319
34,13-312,"John Hall, 27, an Aryan Brotherhood member and inmate at the Federal Correctional Institution (FCI) in Seagoville, Texas, was sentenced today by U.S. District Judge Reed O’Connor after pleading guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act stemming from his assault of a fellow inmate, whom he believed to be gay, the Department of Justice announced. Hall assaulted his fellow inmate with a dangerous weapon, causing bodily injury to the victim on Dec. 20, 2011. Hall was sentenced to serve 71 months in prison to be served consecutively with the sentence he is currently serving. The assault occurred on Dec. 20, 2011, inside the FCI Seagoville when Hall targeted and attacked the victim, a fellow inmate, because he believed the victim was gay or involved in a sexual relationship with another male inmate. Hall repeatedly punched, kicked and stomped on the victim’s face with his shod feet, a dangerous weapon, while yelling a homophobic slur. The victim lost consciousness during the assault and suffered multiple lacerations to his face. The victim also sustained a fractured eye socket, lost a tooth, fractured other teeth and was treated at a hospital for the injuries he sustained during Hall’s unprovoked attack. Hall pleaded guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act on Nov. 8, 2012. “Brutality and violence based on sexual orientation has no place in a civilized society,” said Thomas E. Perez, Assistant Attorney General for the Civil Rights Division. “The Justice Department is committed to using all the tools in our law enforcement arsenal, including the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act, to prosecute acts motivated by hate.” “This prosecution sends a clear message that this office, in partnership with attorneys in the department’s Civil Rights Division, will prioritize and aggressively prosecute hate crimes and others civil rights violations in North Texas,” said U.S. Attorney Sarah R. Saldaña of the Northern District of Texas. This case was investigated by the FBI Dallas Division. The case was prosecuted by Assistant U.S. Attorney Errin Martin and Trial Attorney Adriana Vieco of the Civil Rights Division.",0.300


# WE ARE GETTING SOMETHING DIFFERENT, CHECK ABOVE ^^^

D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [16]:
# Find the mean compound score for each of the three topics in topics_clean
mean_compound = doj_subset_wscores.groupby('topics_clean')['neg'].agg('mean')
mean_compound

topics_clean
Civil Rights              0.099525
Hate Crimes               0.177472
Project Safe Childhood    0.114476
Name: neg, dtype: float64

# ALSO GETTING OFF ANS PLS CHECK ABOVE ^

**Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)**

# CAN SOMEONE PLS FILL IN and ans PART E plz right here <3

# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/

In [17]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]

In [18]:
# A

# Initialize Snowball Stemmer
porter = SnowballStemmer(language='english')

# Creates custom list of stop words
list_stopwords = stopwords.words("english")
list_stopwords_new = list_stopwords + custom_doj_stopwords

# Does the following preprocessing steps:

# Converts the words to lowercase
# HOW TO DO THIS: Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
# Only retains alpha words (so removes digits and punctuation)
# Only retains words 4 characters or longer
# Uses the snowball stemmer from nltk to stem
# Returns a joined preprocessed string

# Define function
def preprocess_data(text):
    
    # Converts words to lowercase
    corpus_lower = text.lower()
    
    nostop = [word for word in wordpunct_tokenize(corpus_lower) if word not in list_stopwords_new]
    clean = [porter.stem(word) for word in nostop if word.isalpha() and len(word) >= 4]
    clean_str = " ".join(clean)

    return(clean_str)

# HOW TO ADD CUSTOM STOPWORDS TO YOUR CODE CELL??!?!?!?!?!?!?!?!?!?

In [19]:
# B
# List comprehension to execute function
preprocessed = [preprocess_data(str) for str in doj_subset_wscores.contents]

# Create a new column in the data called processed_text
doj_subset_wscores['processed_text'] = preprocessed

In [20]:
## your code showing the examples
# C
# Print the id, contents, and processed_text columns for the following press releases:
two_cases = doj_subset_wscores[doj_subset_wscores['id'].isin(['16-718', '16-217'])]
two_cases[['id', 'contents', 'processed_text']]

,id,contents,processed_text
632,16-718,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment",nine count indict unseal today mississippi correct offic charg beat inmat third charg help cover indict charg lawardrick marsher robert sturdiv offic mississippi state penitentiari parchman mississippi beat includ kick punch throw victim ground marsher sturdiv charg violat right convict prison free cruel unusu punish sturdiv also charg fail interven marsher punch beat indict alleg action involv danger weapon result bodili injuri victim third offic deont pate charg along marsher sturdiv conspir cover beat indict alleg three offic submit fals report three lie convict marsher sturdiv face maximum sentenc year prison excess forc charg three offic face five year prison conspiraci fals statement charg year prison fals report charg indict mere accus defend presum innoc unless proven guilti investig jackson cooper mississippi correct prosecut robert coleman northern mississippi dana mulhaus crimin section marsher indict
313,16-217,"The Justice Department has reached a comprehensive settlement agreement with the city of Miami and the Miami Police Department (MPD) resolving the Justice Department’s investigation of officer-involved shootings by MPD officers, announced Principal Deputy Assistant Attorney General Vanita Gupta, head of the Justice Department’s Civil Rights Division and U.S. Attorney Wifredo A. Ferrer of the Southern District of Florida. The settlement, which was approved by Miami’s city commission today and will go into effect when the agreement is signed by all parties, resolves claims stemming from the Justice Department’s investigation into officer-involved shootings by MPD officers, which was conducted under the Violent Crime Control and Law Enforcement Act of 1994. The investigation’s findings, issued in July 2013, identified a pattern or practice of excessive use of force through officer-involved shootings in violation of the Fourth Amendment of the Constitution. The city’s compliance with the settlement will be monitored by an independent reviewer, former Tampa, Florida, Police Chief Jane Castor. Under the settlement agreement, the city will implement comprehensive reforms to ensure constitutional policing and support public trust. The settlement agreement is designed to minimize officer-inv

# We are getting some errors, most likely from the function above YIKES ^^ above 
# (probs from part A)

## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.


In [21]:
def create_dtm(list_of_strings, metadata):
    
    vectorizer = CountVectorizer(lowercase = True)
    
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), columns=vectorizer.get_feature_names_out())
    
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    
    return(dtm_dense_named_withid)

In [22]:
# your code here
# A
# Metadata list
metadata_list = ['id', 'compound', 'topics_clean']

# Create a document-term matrix using the preprocessed press releases
doj_dtm = create_dtm(list_of_strings = doj_subset_wscores['processed_text'],
                    metadata = doj_subset_wscores[metadata_list])

doj_dtm

,index,id,compound,topics_clean,aaron,abandon,abbat,abbi,abbott,abdomen,...,zane,zealand,zealous,zeeman,zero,zionism,zobel,zone,zunggeemog,zwengel
0,13,14-248,-0.9950,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,34,13-312,-0.9983,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,632,16-718,-0.9964,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,22,11-626,-0.9986,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,594,10-1194,-0.9990,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,392,16-539,0.9854,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
713,324,17-132,0.9794,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
714,346,17-003,0.9766,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
715,72,17-271,0.9909,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# Initialize vectorizer
vectorizer = CountVectorizer(lowercase=True)

# B
# Calculate 95%ile of compound sentiment scores (most positive)
top_5 = doj_subset_wscores['compound'].quantile(.95)

# Select press releases with compound sentiment scores in top 5%
most_pos_sent = doj_subset_wscores[doj_subset_wscores['compound'] >= top_5]

# Transform selected press releases into DTM
dtm_pos = vectorizer.fit_transform(most_pos_sent['processed_text'])

# Sum frequencies of each word in DTM
pos_freq = dtm_pos.sum(axis = 0)

# List of positive words
word_list_pos = vectorizer.get_feature_names_out()

# Top 10 words with positive sentiment
print("Top 10 words with positive sentiment:")
for idx in pos_freq.argsort()[0, -10:]:
    print(word_list_pos[idx])

# C
# Calculate 5%ile of compound sentiment scores (most negative)
bottom_5 = doj_subset_wscores['compound'].quantile(.05)

# Press releases in the bottom 5%
neg_sent = doj_subset_wscores[doj_subset_wscores['compound'] >= bottom_5]

# Make into DTM
dtm_neg = vectorizer.fit_transform(neg_sent['processed_text'])

# Sum frequencies of each word
neg_freq = dtm_neg.sum(axis = 0)

# Top 10 words with negative sentiment
word_list_neg = vectorizer.get_feature_names_out()
print("Top 10 words with negative sentiment:")
for idx in neg_freq.argsort()[0, -10:]:
    print(word_list_neg[idx])

Top 10 words with positive sentiment:
[['servic' 'general' 'student' 'settlement' 'communiti' 'disabl' 'ensur'
  'state' 'enforc' 'agreement']]
Top 10 words with negative sentiment:
[['general' 'year' 'charg' 'state' 'said' 'sentenc' 'prosecut' 'victim'
  'feder' 'child']]


In [24]:
# D
# Print the top 10 words for press releases in each of the three topics_clean
topics_3 = doj_subset_wscores.topics_clean.unique()

for topic in topics_3:

    # Filter for topics
    topic_new = doj_subset_wscores[doj_subset_wscores['topics_clean'] == topic]

    # Create dtm 
    dtm_topic = vectorizer.fit_transform(topic_new['processed_text'])

    # Sum frequency of each word
    freq_topic = dtm_topic.sum(axis = 0)

    # Convert to DataFrame
    topic_df = pd.DataFrame(freq_topic, columns=vectorizer.get_feature_names_out())
    
    # Sorting from most frequent to least frequent
    top_words = topic_df.sum().sort_values(ascending=False)
    
    # Print the top 10 words for the current topic
    print(f"\nTop 10 words for press releases in topic: {topic}")
    print(top_words.head(10))


Top 10 words for press releases in topic: Hate Crimes
victim      591
crime       557
hate        524
defend      484
prosecut    478
charg       463
sentenc     455
american    451
feder       432
guilti      430
dtype: int64

Top 10 words for press releases in topic: Civil Rights
offic        637
hous         633
discrimin    616
enforc       544
disabl       532
said         497
feder        479
violat       477
state        452
court        414
dtype: int64

Top 10 words for press releases in topic: Project Safe Childhood
child          1022
exploit         701
sexual          572
safe            479
childhood       474
project         472
pornographi     452
children        423
crimin          405
prosecut        374
dtype: int64


## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [25]:
# A - Create topic model from preprocessed strings
# Retokenize
text_tokens = [word_tokenize(text) for text in doj_subset_wscores.processed_text]

# Create dictionary
text_proc_dict = corpora.Dictionary(text_tokens)

# Filter dictionary w/ 2% bounds
text_proc_dict.filter_extremes(no_below = round(doj_subset_wscores.shape[0]*0.02),
                             no_above = round(doj_subset_wscores.shape[0]*0.98))

# Create corpus from dictionary
corpus_fromdict_proc = [text_proc_dict.doc2bow(text) 
                       for text in text_tokens]

# Estimate model
n_topics = 3
lda_model = gensim.models.LdaModel(corpus_fromdict_proc, num_topics=n_topics, id2word=text_proc_dict, passes=10)
# #lda_model = gensim.models.ldamodel.LdaModel(corpus_fromdict_proc, 
#                                               num_topics = n_topics, 
#                                               id2word=text_proc_dict, 
#                                               passes=6, alpha = 'auto',
#                                               per_word_topics = True, 
#                                               random_state = 91988)

# topics = lda_model.print_topics(num_words = 15)
# for topic in topics:
#     print(topic)

# B - print the top 15 words
for topic_id, topic_words in lda_model.print_topics(num_words = 15):
    print(f"Topic {topic_id + 1}: {topic_words}")

Topic 1: 0.026*"child" + 0.018*"exploit" + 0.017*"sexual" + 0.014*"victim" + 0.013*"sentenc" + 0.013*"crimin" + 0.013*"prosecut" + 0.012*"safe" + 0.012*"project" + 0.012*"childhood" + 0.011*"pornographi" + 0.011*"children" + 0.010*"offic" + 0.010*"year" + 0.009*"guilti"
Topic 2: 0.013*"discrimin" + 0.011*"hous" + 0.011*"disabl" + 0.010*"enforc" + 0.009*"offic" + 0.008*"agreement" + 0.008*"said" + 0.008*"state" + 0.008*"court" + 0.008*"violat" + 0.007*"feder" + 0.007*"requir" + 0.007*"general" + 0.007*"settlement" + 0.006*"provid"
Topic 3: 0.012*"charg" + 0.012*"victim" + 0.012*"defend" + 0.011*"crime" + 0.011*"prosecut" + 0.011*"hate" + 0.011*"sentenc" + 0.011*"said" + 0.011*"feder" + 0.010*"guilti" + 0.010*"indict" + 0.010*"american" + 0.009*"year" + 0.008*"african" + 0.008*"today"


## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)


In [26]:
## your code here to get doc-level topic probabilities 
# A
topic_prob_bydoc = [lda_model.get_document_topics(item, minimum_probability=0) for item in corpus_fromdict_proc]

assert len(topic_prob_bydoc) == len(doj_subset_wscores), "The length of topic probabilities list does not match the number of rows in doj_subset_wscores dataframe"

In [32]:
doj_subset_wscores.shape

(717, 25)

In [36]:
np.argmax([1,3,2])

1

In [34]:
topic_prob_bydoc
# make a function that iterates through the sublist and see if you can figure out which one gets the highest number

[[(0, 0.07645117), (1, 0.0057064733), (2, 0.9178424)],
 [(0, 0.15268771), (1, 0.0026279662), (2, 0.8446843)],
 [(0, 0.21017163), (1, 0.0035041792), (2, 0.78632414)],
 [(0, 0.0023918315), (1, 0.0023418332), (2, 0.9952664)],
 [(0, 0.0019146346), (1, 0.0018721769), (2, 0.9962132)],
 [(0, 0.0036185677), (1, 0.0035110079), (2, 0.99287045)],
 [(0, 0.013586157), (1, 0.9703147), (2, 0.016099142)],
 [(0, 0.24460426), (1, 0.004413338), (2, 0.7509824)],
 [(0, 0.011585373), (1, 0.002476054), (2, 0.9859386)],
 [(0, 0.0021685807), (1, 0.0017200222), (2, 0.9961114)],
 [(0, 0.11900537), (1, 0.09421623), (2, 0.7867784)],
 [(0, 0.0048661055), (1, 0.0047382005), (2, 0.9903957)],
 [(0, 0.0035489176), (1, 0.005880447), (2, 0.99057066)],
 [(0, 0.041322388), (1, 0.0020104009), (2, 0.9566672)],
 [(0, 0.0023485299), (1, 0.0022820968), (2, 0.9953694)],
 [(0, 0.0039466294), (1, 0.0036040645), (2, 0.99244934)],
 [(0, 0.3551811), (1, 0.0051283035), (2, 0.63969064)],
 [(0, 0.02330604), (1, 0.0014382371), (2, 0.9752

In [29]:
## your code here to add those topic probabilities to the dataframe
# B

one_list_tup = topic_prob_bydoc[0]
one_list_tup

# create a long form dataframe by flattening the list
topic_prob_bydoc_long = pd.DataFrame([t for lst in topic_prob_bydoc for t in lst],
                                     columns = ['topic', 'probability'])

# add id var for the number of topics
topic_prob_bydoc_long['text'] = list(np.concatenate([[text] * 
                                    n_topics for text in doj_subset_wscores.processed_text]).flat)

# pivot to wide format
topic_prob_bydoc_wide = pd.pivot_table(topic_prob_bydoc_long, index = ['text'],
                        columns = ['topic']).reset_index().reset_index(drop = True)
topic_prob_bydoc_wide.columns = ['text'] + ["topic_" + str(i) for i in np.arange(0, n_topics)]

# merge w/ original data set
doj_subset_wscores = pd.merge(topic_prob_bydoc_wide, doj_subset_wscores, left_on = 'text', right_on = 'processed_text')
doj_subset_wscores
# # Identify topic columns
# topic_columns = [col for col in doj_subset_wscores.columns if col.startswith('topic_')]

# # Find the column with the highest probability for each row
# doj_subset_wscores['top_topic'] = doj_subset_wscores[topic_columns].idxmax(axis=1)

# # If you want to get the topic number as an integer instead of column name:
# # Extract the topic number from the column name
# doj_subset_wscores['top_topic'] = doj_subset_wscores['top_topic'].str.replace('topic_', '').astype(int)

# # create indicator for top topics
# Identify topic columns


# Create TOP TOPIC column -- HOW TO DO?!?!??!!??!?!?!

[(0, 0.07645117), (1, 0.0057064733), (2, 0.9178424)]

,text,topic_0,topic_1,topic_2,text_x,topic_0_x,topic_1_x,topic_2_x,text_y,topic_0_y,...,contents,date,topics_clean,components_clean,index,neg,neu,pos,compound,processed_text
0,act general jocelyn samuel gregori davi southern mississippi announc today feder grand juri indict john loui blalack brandon mississippi sarah adelia grave crystal spring mississippi robert henri rice brandon shelbi brook richard pearl mississippi alleg role conspiraci commit feder hate crime african american peopl jackson mississippi blalack grave richard addit charg racial motiv hate crime result death victim truck blalack rice charg addit racial motiv hate crime involv alleg assault carri firearm relat assault grave richard addit charg solicit other commit hate crime african american grave charg make fals statement defend deryl paul dedmon john aaron rice dylan wade butler william kirk montgomeri jonathan kyle gaskamp joseph dominick brandon previous enter guilti plea connect role offens indict alleg begin spring defend other conspir anoth harass assault african american peopl around jackson area accord indict numer occas conspir use danger weapon includ beer bottl sling shot motor vehicl caus attempt caus bodili injuri african american peopl conspir alleg specif target african american peopl believ homeless influenc alcohol believ individu would less like report assault conspir would often boast racial motiv assault indict detail sever assault includ fatal assault victim intent defend face statutori maximum sentenc life prison result cooper effort southern mississippi hind counti investig jackson jackson polic prosecut sheldon beer deputi chief paig fitzgerald glenda hayn southern mississippi charg forth indict mere accus defend presum innoc proven guilti,0.076451,0.005706,0.917842,act general jocelyn samuel gregori davi southern mississippi announc today feder grand juri indict john loui blalack brandon mississippi sarah adelia grave crystal spring mississippi robert henri rice brandon shelbi brook richard pearl mississippi alleg role conspiraci commit feder hate crime african american peopl jackson mississippi blalack grave richard addit charg racial motiv hate crime result death victim truck blalack rice charg addit racial motiv hate crime involv alleg assault carri firearm relat assault grave richard addit charg solicit other commit hate crime african american grave charg make fals statement defend deryl paul dedmon john aaron rice dylan wade butler william kirk montgomeri jonathan kyle gaskamp joseph dominick brandon previous enter guilti plea connect role offens indict alleg begin spring defend other conspir anoth harass assault african american peopl around jackson area accord indict numer occas conspir use danger weapon includ beer bottl sling shot motor vehicl caus attempt caus bodili injuri african american peopl conspir alleg specif target african american peopl believ homeless influenc alcohol believ individu would less like report assault conspir would often boast racial motiv assault indict detail sever assault includ fatal assault victim intent defend face statutori maximum sentenc life prison result cooper effort southern mississippi hind counti investig jackson jackson polic prosecut sheldon beer deputi chief paig fitzgerald glenda hayn southern mississippi charg forth indict mere accus defend presum innoc proven guilti,0.076451,0.005706,0.917842,act general jocelyn samuel gregori davi southern mississippi announc today feder grand juri indict john loui blalack brandon mississippi sarah adelia grave crystal spring mississippi robert henri rice brandon shelbi brook richard pearl mississippi alleg role conspiraci commit feder hate crime african american peopl jackson mississippi blalack grave richard addit charg racial motiv hate crime result death victim truck blalack rice charg addit racial motiv hate crime involv alleg assault carri firearm relat assault grave richard addit charg solicit other commit hate crime african american grav

In [ ]:
## create indicator for listing's top topic
topic_wmeta['toptopic'] = topic_wmeta[[col for col in topic_wmeta.columns if 
                                    "topic_" in col]].idxmax(axis=1)
topic_wmeta.sample(n = 5, random_state = 555)

## group by topic and find mean price
topic_wmeta.groupby('toptopic').agg({'price_rawdata': np.mean})

## group by borough and topic -- higher price for some also reflects
## diff borough composition
topic_wmeta.groupby(['toptopic', 
                    'neighbourhood_group']).agg({'price_rawdata': np.mean})

In [26]:
## your code here to summarize the topic proportions for each of the topics_clean 

**D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)**

# FILL IN

# 3. Extend the analysis from unigrams to bigrams (10 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [37]:
# A

# Write bigrams function
def create_bigram_one(text):
    

# Create new column and combine words with "_"
doj_subset_wscore['processed_text_bigrams'] =

,text,topic_0,topic_1,topic_2,text_x,topic_0_x,topic_1_x,topic_2_x,text_y,topic_0_y,...,topics_clean,components_clean,index,neg,neu,pos,compound,processed_text,top_topic,processed_text_bigrams
0,act general jocelyn samuel gregori davi southern mississippi announc today feder grand juri indict john loui blalack brandon mississippi sarah adelia grave crystal spring mississippi robert henri rice brandon shelbi brook richard pearl mississippi alleg role conspiraci commit feder hate crime african american peopl jackson mississippi blalack grave richard addit charg racial motiv hate crime result death victim truck blalack rice charg addit racial motiv hate crime involv alleg assault carri firearm relat assault grave richard addit charg solicit other commit hate crime african american grave charg make fals statement defend deryl paul dedmon john aaron rice dylan wade butler william kirk montgomeri jonathan kyle gaskamp joseph dominick brandon previous enter guilti plea connect role offens indict alleg begin spring defend other conspir anoth harass assault african american peopl around jackson area accord indict numer occas conspir use danger weapon includ beer bottl sling shot motor vehicl caus attempt caus bodili injuri african american peopl conspir alleg specif target african american peopl believ homeless influenc alcohol believ individu would less like report assault conspir would often boast racial motiv assault indict detail sever assault includ fatal assault victim intent defend face statutori maximum sentenc life prison result cooper effort southern mississippi hind counti investig jackson jackson polic prosecut sheldon beer deputi chief paig fitzgerald glenda hayn southern mississippi charg forth indict mere accus defend presum innoc proven guilti,0.870248,0.123368,0.006385,act general jocelyn samuel gregori davi southern mississippi announc today feder grand juri indict john loui blalack brandon mississippi sarah adelia grave crystal spring mississippi robert henri rice brandon shelbi brook richard pearl mississippi alleg role conspiraci commit feder hate crime african american peopl jackson mississippi blalack grave richard addit charg racial motiv hate crime result death victim truck blalack rice charg addit racial motiv hate crime involv alleg assault carri firearm relat assault grave richard addit charg solicit other commit hate crime african american grave charg make fals statement defend deryl paul dedmon john aaron rice dylan wade butler william kirk montgomeri jonathan kyle gaskamp joseph dominick brandon previous enter guilti plea connect role offens indict alleg begin spring defend other conspir anoth harass assault african american peopl around jackson area accord indict numer occas conspir use danger weapon includ beer bottl sling shot motor vehicl caus attempt caus bodili injuri african american peopl conspir alleg specif target african american peopl believ homeless influenc alcohol believ individu would less like report assault conspir would often boast racial motiv assault indict detail sever assault includ fatal assault victim intent defend face statutori maximum sentenc life prison result cooper effort southern mississippi hind counti investig jackson jackson polic prosecut sheldon beer deputi chief paig fitzgerald glenda hayn southern mississippi charg forth indict mere accus defend presum innoc proven guilti,0.870248,0.123368,0.006385,act general jocelyn samuel gregori davi southern mississippi announc today feder grand juri indict john loui blalack brandon mississippi sarah adelia grave crystal spring mississippi robert henri rice brandon shelbi brook richard pearl mississippi alleg role conspiraci commit feder hate crime african american peopl jackson mississippi blalack grave richard addit charg racial motiv hate crime result death victim truck blalack rice charg addit racial motiv hate crime involv alleg assault carri firearm relat assault grave richard addit charg solicit other commit hate crime af

C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the (1) dimensions of the `dtm` matrix from question 2.2  and (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2

In [28]:
# your code here

# 4. Optional extra credit (2 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [29]:
# your code here 